Dado un conjunto de n objetos cuyos tamaños son {T1, T2,··· , Tn}, con Ti ∈ (0,1], se debe empaquetarlos usando la mínima cantidad de envases de capacidad 1. 

Solución por backtracking/fuerza bruta.

In [55]:
# to use deepcopy
from copy import deepcopy
from math import inf

def _empaquetar_back(items, paquetes, tabs=0, len_min=inf):
    if len(items) == 0:
        # print("\t"*tabs, "items:", items)
        # print("\t"*tabs, "paquetes:", paquetes)
        return paquetes
    
    # tomamos el primer item
    item = items.pop(0)

    # agrego un nuevo paquete
    if paquetes == []:
        paquetes.append([item])
        return _empaquetar_back(items.copy(), deepcopy(paquetes), tabs+1)

    # hay que empaquetar el item
    # puede ser en un paquete existente
    # o en un paquete nuevo
    opciones = []
    # print("\t"*tabs, "item:", item)
    # print("\t"*tabs, "paquetes:", paquetes)
    for i in range(len(paquetes)):
        if sum(paquetes[i]) + item <= 1:
            paquetes_copia = deepcopy(paquetes)
            paquetes_copia[i].append(item)
            opcion = _empaquetar_back(items.copy(), paquetes_copia, tabs+1, len_min)
            # if len(opcion) < len_min:
            #     len_min = len(opcion)
            #     opciones = [opcion]

    
    # if len(opciones) + 1 <= len_min:
    paquetes_copia = deepcopy(paquetes)
    paquetes_copia.append([item])
    opciones.append(_empaquetar_back(items.copy(), paquetes_copia, tabs+1, len_min))
            
    # print("\t"*tabs, "opciones:", opciones)
    min = opciones[0]

    for opcion in opciones:
        if len(opcion) < len(min):
            min = opcion

    # print("\t"*tabs, "min:", min)

    return min
            
def empaquetar_back(items):
    paquetes = []
    return _empaquetar_back(items, paquetes)

In [56]:
items =  [0.5, 0.4, 0.3, 0.2, 0.1]
print(empaquetar_back(items))

[[0.5], [0.4], [0.3], [0.2], [0.1]]


In [57]:
# {0.4; 0.8; 0.5; 0.1; 0.7; 0.6; 0.1; 0.4; 0.2; 0.2}
items = [0.4, 0.8, 0.5, 0.1, 0.7, 0.6, 0.1, 0.4, 0.2, 0.2]
print(empaquetar_back(items))

[[0.4], [0.8], [0.5], [0.1], [0.7], [0.6], [0.1], [0.4], [0.2], [0.2]]
